In [ ]:
import numpy as np
import pandas as pd
from scipy.ndimage.interpolation import shift
from sklearn.metrics import mean_absolute_error
from matplotlib import pyplot as plt

In [ ]:
# DF = pd.read_csv(r'train.csv')

In [ ]:
# for val in DF.Asset_ID.unique():
#     print(val)
#     DF[DF.Asset_ID==val].sort_values(by=['timestamp'],ascending=True).to_csv(r'splitdata\Asset'+str(val)+'.csv',index=None)

In [ ]:
#del DF

In [ ]:
_ = pd.read_csv(r'splitdata\Asset1.csv')

In [ ]:
DF = _.copy(deep=True)
del _

In [ ]:
DF[['timestamp','Asset_ID','Close','Target']]

In [ ]:
np.array(DF.Close)

In [ ]:
shift(DF.Close,-16,cval=np.NaN)

In [ ]:
DF['TargetPrice16'] = shift(DF.Close,-16,cval=np.NaN)
DF['TargetPrice1'] = shift(DF.Close,-1,cval=np.NaN)

In [ ]:
DF

In [ ]:
DF['Rt'] = np.log10(np.divide(DF['TargetPrice16'],DF['TargetPrice1']))
#DF['Rt'] = np.divide(DF['TargetPrice16'],DF['TargetPrice1'])

In [ ]:
DF['Rt']

In [ ]:
def moving_average(a, n=3):
    a = np.nan_to_num(a,nan=0)
    ret = np.cumsum(a)
    i=0
    finalArr = []
    for v in ret:
        if(i-n<0):
            finalArr.append(v/(i+1))
        else:
            finalArr.append((v-ret[i-n])/n)
        i+=1
    print(ret)
    return finalArr

In [ ]:
N=3750
DF['Mt'] = moving_average(np.array(DF['Rt']),n=N)

In [ ]:
DF['Mt']

In [ ]:
DF

In [ ]:
def ResidualizeMarket(df, mktColumn, window):
  if mktColumn not in df.columns:
    return df

  mkt = df[mktColumn]

  num = df.multiply(mkt.values, axis=0).rolling(window).mean().values  #numerator of linear regression coefficient
  denom = mkt.multiply(mkt.values, axis=0).rolling(window).mean().values  #denominator of linear regression coefficient
  beta = np.nan_to_num( num.T / denom, nan=0., posinf=0., neginf=0.)  #if regression fell over, use beta of 0

  resultRet = df - (beta * mkt.values).T  #perform residualization
  resultBeta = 0.*df + beta.T  #shape beta

  return resultRet.drop(columns=[mktColumn]), resultBeta.drop(columns=[mktColumn])

In [ ]:
M,N = ResidualizeMarket(DF,'Mt',3750)
DF['Rt'] = M['Rt'].copy()
DF['B'] = N['Rt'].copy()

In [ ]:
N

In [ ]:
DF['TargetCalc'] = DF['Rt'] - DF['B']*DF['Mt']

In [ ]:
DF[['Target','TargetCalc']]

In [ ]:
DF['DiffError'] = np.abs(DF['TargetCalc'] - DF['Target'])

In [ ]:
DF[['TargetCalc','Target','DiffError']].head(20)

In [ ]:
DF['DiffError'].describe()

In [ ]:
#noise = np.random.normal(0, 1, (1000, ))
import scipy.stats as stats
TargetCalcDist = np.nan_to_num(DF['DiffError'], nan=0.0, posinf=0.0, neginf=0.0)
#density = stats.gaussian_kde(TargetCalcDist)
# n, xtc, _ = plt.hist(TargetCalcDist, bins=np.linspace(-0.1, 0.1, 1000), 
#                    histtype=u'step', density=True)  
#plt.plot(density(xtc),label=('Target Error Calc'))
plt.hist(TargetCalcDist,bins=10,label='TargetErrorDistribution')
#---
plt.yscale('log')
plt.legend()
plt.show()